- 각 구별 가장 싼 주유소와 비싼 주유소

In [1]:
import pandas as pd
import requests
import numpy as np
import json, folium
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../05.Crawling/서울-주유소(220418).csv')
df.head()

,상호,주소,가격,셀프,상표,구
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,Y,현대오일뱅크,강동구
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1983,N,현대오일뱅크,강동구
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1995,N,GS칼텍스,강동구
3,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),2015,Y,GS칼텍스,강동구
4,대성석유(주)길동주유소,서울 강동구 천호대로 1168,2025,N,GS칼텍스,강동구


- 각 구별 비싼 주유소

In [3]:
idx = df.groupby('구')['가격'].transform(max) == df['가격']
exp = df[idx]
exp.head()

,상호,주소,가격,셀프,상표,구
13,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,N,S-OIL,강동구
32,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,N,현대오일뱅크,동대문구
42,살피재주유소,서울 동작구 상도로 334,2135,N,SK에너지,동작구
54,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,N,S-OIL,마포구
69,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,N,S-OIL,서대문구


- 각 구별 싼 주유소

In [4]:
idx = df.groupby('구')['가격'].transform(min) == df['가격']
cheap = df[idx]
cheap.head()

,상호,주소,가격,셀프,상표,구
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,Y,현대오일뱅크,강동구
14,대성산업(주)동마장주유소,서울 동대문구 고산자로 405 (용두동),1935,Y,GS칼텍스,동대문구
15,대성산업㈜청량리주유소,서울 동대문구 왕산로 289 (청량리동),1935,Y,GS칼텍스,동대문구
16,동서울고속주유소,서울 동대문구 한천로 414,1935,Y,SK에너지,동대문구
17,풍한주유소,서울 동대문구 안암로 168,1935,N,SK에너지,동대문구


In [17]:
mn = df.groupby('구')[['가격']].mean()
mn.head()

,가격
구,
강남구,2131.848485
강동구,2072.928571
강북구,1960.461538
강서구,1984.156250
관악구,2004.214286


- 각 주유소 주소의 위도 경도 구하기

In [5]:
import requests
from urllib.parse import quote

In [6]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [7]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울 강남구 남부순환로 2718 (도곡2동)'
url = f'{local_url}?query={quote(addr)}'

In [8]:
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [9]:
result = requests.get(url, headers=header).json()

In [10]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
print(lng, lat)

127.043319375098 37.4854150814069


    - 각 구별 비싼 주유소 위도, 경도

In [11]:
lng_list, lat_list = [], []
for i in exp.주소:
    url = f'{local_url}?query={quote(i)}'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))
exp['위도'] = lat_list
exp['경도'] = lng_list
exp.reset_index(inplace=True)
exp.head()

,index,상호,주소,가격,셀프,상표,구,위도,경도
0,13,광성주유소,서울 강동구 올림픽로 673 (천호동),2298,N,S-OIL,강동구,37.541577,127.124242
1,32,한영주유소 (한화),서울 동대문구 망우로 38 (휘경동),2190,N,현대오일뱅크,동대문구,37.588425,127.057395
2,42,살피재주유소,서울 동작구 상도로 334,2135,N,SK에너지,동작구,37.498360,126.951389
3,54,마포주유소,서울 마포구 마포대로 186 (공덕동),2298,N,S-OIL,마포구,37.550595,126.955637
4,69,문화사랑주유소,서울 서대문구 세검정로 87 (홍제동),2059,N,S-OIL,서대문구,37.595442,126.948477


    - 각 구별 싼 주유소 위도, 경도

In [12]:
lng_list, lat_list = [], []
for i in cheap.주소:
    url = f'{local_url}?query={quote(i)}'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))
cheap['위도'] = lat_list
cheap['경도'] = lng_list
cheap.reset_index(inplace=True)
cheap.head()

,index,상호,주소,가격,셀프,상표,구,위도,경도
0,0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,Y,현대오일뱅크,강동구,37.536417,127.149372
1,14,대성산업(주)동마장주유소,서울 동대문구 고산자로 405 (용두동),1935,Y,GS칼텍스,동대문구,37.575865,127.037719
2,15,대성산업㈜청량리주유소,서울 동대문구 왕산로 289 (청량리동),1935,Y,GS칼텍스,동대문구,37.586470,127.051603
3,16,동서울고속주유소,서울 동대문구 한천로 414,1935,Y,SK에너지,동대문구,37.597544,127.068039
4,17,풍한주유소,서울 동대문구 안암로 168,1935,N,SK에너지,동대문구,37.589211,127.035993


- 각 주유소 지도위에 표시하기

In [13]:
map = folium.Map(
    location=[exp.위도.mean(), exp.경도.mean()], zoom_start=12
)
for i in exp.index:
    folium.Marker(
        location=[exp.위도[i], exp.경도[i]],
        popup = folium.Popup(exp.주소[i], max_width=200),
        tooltip=exp.상호[i],
        icon=folium.Icon(color='red')
    ).add_to(map)

for i in cheap.index:
    folium.Marker(
        location=[cheap.위도[i], cheap.경도[i]],
        popup = folium.Popup(cheap.주소[i], max_width=200),
        tooltip=cheap.상호[i],
        icon=folium.Icon(color='green')
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">각 구별 가격이 싼/비싼 주유소</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

- Choropleth 위에 표시하기

In [19]:
filename = '../04.지도시각화/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [20]:
map = folium.Map(
    location=[exp.위도.mean(), exp.경도.mean()], zoom_start=12
)
for i in exp.index:
    folium.Marker(
        location=[exp.위도[i], exp.경도[i]],
        popup = folium.Popup(exp.주소[i], max_width=200),
        tooltip=exp.상호[i],
        icon=folium.Icon(color='red')
    ).add_to(map)

for i in cheap.index:
    folium.Marker(
        location=[cheap.위도[i], cheap.경도[i]],
        popup = folium.Popup(cheap.주소[i], max_width=200),
        tooltip=cheap.상호[i],
        icon=folium.Icon(color='green')
    ).add_to(map)

folium.Choropleth(
    geo_data=geo_str, 
    data=mn.가격,
    columns=[mn.index, mn.가격],
    fill_color='PuRd',
    key_on='feature.id'
)
title_html = '<h3 align="center" style="font-size:20px">각 구별 가격이 싼/비싼 주유소</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map